## Task 1: inspect prior distributions

In [ ]:
# COLAB users: load simulations and data
system("ls", TRUE)

system("wget https://raw.githubusercontent.com/manolofperez/Physalia_MLPopGen/refs/heads/main/2025/Day1/Notebooks/Day1c/mosquito-task1.csv", TRUE)

system("wget https://raw.githubusercontent.com/manolofperez/Physalia_MLPopGen/refs/heads/main/2025/Day1/Notebooks/Day1c/mosquito-task2.csv", TRUE)

system("wget https://raw.githubusercontent.com/manolofperez/Physalia_MLPopGen/refs/heads/main/2025/Day1/Notebooks/Day1c/mosquito-observed.csv", TRUE)

# Then you just need to load it, as the files will be in the root directory of your Colab.

In [ ]:
sims <- read.csv("mosquito-task1.csv", head=T)

# check prior distributions
x11()
par(mfrow=c(2,2))
hist(sims$N1)
hist(sims$N2)
hist(sims$T_split)
hist(sims$mig)


## Task 2: perform ABC

In [ ]:
sims <- read.csv("mosquito-task2.csv", head=T)

# check prior distributions
x11()
par(mfrow=c(2,2))
hist(sims$N1)
hist(sims$N2)
hist(sims$T_split)
hist(sims$mig)


In [ ]:
# remove simulations with NaN for some summary stats!

# find useful summary stats which correlate with T_split
cor(sims$Fst, sims$T_split)
cor(sims$dxy, sims$T_split)
cor(sims$segsites1, sims$T_split)
cor(sims$segsites2, sims$T_split)
cor(sims$pi1, sims$T_split)
cor(sims$pi2, sims$T_split)
cor(sims$tajima1, sims$T_split)
cor(sims$tajima2, sims$T_split)


In [ ]:
# load observed summary stats
obs <- read.csv("mosquito-observed.csv", head=T)

# check if simulated retained summary stats contain the observed one
quantile(sims$Fst); cat(obs$Fst)
quantile(sims$segsites1); cat(obs$segsites1)
quantile(sims$segsites2); cat(obs$segsites2)


In [ ]:
# merge obs with retained sims to scale them
sumstats <- scale(rbind(obs[c(1,3,4)],sims[,c(5,7,8)]))
head(sumstats)

In [ ]:
# load ABC library
library(abc)

In [ ]:
est <- abc(target=sumstats[1,], param=sims$T_split, sumstat=sumstats[-1,], tol=0.05, method="rejection")

In [ ]:
# check distances in the acceptance region
hist(est$dist)
abline(v=max(est$dist[which(est$region)]), lty=2)

In [ ]:
# posterior distribution
x11()
par(mfrow=c(2,1))
hist(est$unadj.values, freq=FALSE, xlim=range(sims$T_split), col=rgb(0,0,1,1/4), main="Posterior probability", xlab="Split time")

# MAP
map <- mean(est$unadj.values)
abline(v=map, lty=2)

# confidence intervals
hpd <- quantile(x=est$unadj.values, probs=c(0.025,0.975))
abline(v=hpd, lty=3)

# prior
hist(sims$T_split, freq=FALSE, xlim=range(sims$T_split), col=rgb(1,0,0,1/4))
